In [12]:
import time
import configparser
import csv
import os
import traceback

from utility.m21_utility import *

from preprocess.piece import Piece
from preprocess.note import Note
from segmentation import *
from identification.key_identification import (
    determine_key_by_adjacent,
    determine_key_solo,
)
from identification.note_to_chord import find_chords
from identification.result_combination import *
from evaluation import Metric

CONFIG = configparser.ConfigParser()
CONFIG.read(os.path.join(".", "config.ini"))

try:
    DATA_PATH = CONFIG["locations"]["data_path"]
    DATASET_PATH = CONFIG["locations"]["dataset_path"]
    RESULT_PATH = CONFIG["locations"]["result_path"]
    CSV_PATH = CONFIG["locations"]["csv_path"]

    KeyThenChordMode = CONFIG["param"]["KeyThenChord"]
    ExportKey = CONFIG["export"]["keys"]
    ExportChord = CONFIG["export"]["chords"]
except KeyError:
    print("failed to read config.ini, or invalid index specified")
    raise SystemExit


def get_files(filename=None):
    all_scores = [f for f in os.listdir(DATA_PATH + DATASET_PATH) if f.endswith(".mxl")]
    # print(all_scores)
    # RuntimeWarning: invalid value encountered in double_scalars correlation_value = corr_value_numerator / corr_value_denominator
    if filename in all_scores:
        return [filename]
    return all_scores


def export_keys(out_list, dirname, filename):
    path = os.path.join(RESULT_PATH, dirname, filename + ".csv")
    file = open(path, "w", newline="")
    writer = csv.writer(file)

    writer.writerow(("offset", "key"))

    for el in out_list:
        writer.writerow((el[0], el[1]))

    file.close()


def export_chords(out_list, dirname, filename):
    path = os.path.join(RESULT_PATH, dirname, filename + ".csv")
    file = open(path, "w", newline="")
    writer = csv.writer(file)

    for segment in out_list:
        writer.writerow(
            (
                segment["offset"],
                segment["chord"].numeral,
                segment["chord"].scale.__str__(),
                round(segment["score"], 6),
            )
        )

    file.close()

In [17]:
score_file = get_files()[0]

print(">> Currently handling: " + score_file)
piece = Piece(score_file)
stream = piece.score
chordify_stream = piece.chordified
flatten_stream = piece.flattened

time_signature = get_initial_time_signature(flatten_stream)
# key_signature = get_initial_key_signature(flatten_stream) # Scale(key_signature.tonic.name)

# Key segmentation and Identification
# TODO: stream or chordified?
key_segments: Measure_OffsetChroma_dict = key_segmentation(stream)
measures_key = determine_key_by_adjacent(key_segments)
# measures_key = determine_key_solo(key_segmentation(stream))

# Chord segmentation and Identification
notes_in_measures = get_notes_in_measures(chordify_stream)
beat_segments = uniform_segmentation(notes_in_measures, time_signature)
combined_segments = merge_chord_segment(beat_segments)

offset_chord_choices = []
for segment in combined_segments:
    notes_frequencies = [
        (Note(input_str=note_name), value)
        for note_name, value in segment["note_profile"].items()
    ]
    if KeyThenChordMode:
        key_choices = find_scale_in_chord_segment(measures_key, segment)
        possible_key = max(key_choices, key=key_choices.get)
        possible_chords = find_chords(notes_frequencies, possible_key)
        offset_chord_choices.append(
            {"offset": segment["offset"], "chords": possible_chords}
        )

        if ExportKey:
            try:
                key_result = []
                for offs in offset_chord_choices:
                    key_result.append(
                        (
                            offs["offset"],
                            offs["chords"][0]["chord"].scale.__str__(),
                        )
                    )
            except IndexError as ie:
                pass
            # export_keys(key_result, "keys", score_file.removesuffix(".mxl"))
    else:
        offset_chord_choices.append(
            (segment["offset"], find_chords(notes_frequencies))
        )

if ExportChord:
    if KeyThenChordMode:
        measures_key = None
    chord_result = determine_chord(
        keys=measures_key, chords=offset_chord_choices
    )
    # export_chords(chord_result, "chords", score_file.removesuffix(".mxl"))

>> Currently handling: Bach_J.S._Minuet_in_G_Minor_(BWV_Anh.116).mxl


In [20]:
key_segments

{1: {'chroma': array([0. , 0. , 1.5, 0. , 0. , 0. , 0.5, 3. , 0. , 0.5, 0. , 0.5]),
  'offset': 0.0},
 2: {'chroma': array([0., 0., 1., 0., 0., 0., 0., 4., 0., 0., 0., 1.]),
  'offset': 3.0},
 3: {'chroma': array([0. , 0. , 1.5, 0. , 0. , 0. , 0.5, 3. , 0. , 0.5, 0. , 0.5]),
  'offset': 6.0},
 4: {'chroma': array([0., 0., 1., 0., 0., 0., 0., 4., 0., 0., 0., 1.]),
  'offset': 9.0},
 5: {'chroma': array([2. , 0. , 0. , 0. , 2.5, 0. , 0. , 1.5, 0. , 0. , 0. , 0. ]),
  'offset': 12.0},
 6: {'chroma': array([0. , 0. , 2.5, 0. , 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 2. ]),
  'offset': 15.0},
 7: {'chroma': array([2. , 0. , 0.5, 0. , 0. , 0. , 1. , 1. , 0. , 1. , 0. , 0.5]),
  'offset': 18.0},
 8: {'chroma': array([0., 0., 1., 0., 1., 0., 1., 0., 0., 3., 0., 0.]),
  'offset': 21.0},
 9: {'chroma': array([0. , 0. , 1.5, 0. , 0. , 0. , 0.5, 3. , 0. , 0.5, 0. , 0.5]),
  'offset': 24.0},
 10: {'chroma': array([0., 0., 1., 0., 0., 0., 0., 4., 0., 0., 0., 1.]),
  'offset': 27.0},
 11: {'chroma': array

In [19]:
for x in key_segments:
    print(x)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
